In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
df = pd.read_csv("cars_avito.csv", encoding= 'utf-8')
df

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Supprimer les lignes contient des valeurs nulls pour 'Marque' => Une dataset sans valeurs nulls
df = df.dropna(subset= 'Marque')

# Supprimer les colonnes unitiles
df = df.drop(['Secteur', 'Type', 'État'], axis= 1)
df

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Appliquer l'encodage ordinale pour le variable "Premier main"
condition_ctg = ['Oui', 'Non']

condition_encoder = OrdinalEncoder(categories= [condition_ctg])

df['Première main'] = condition_encoder.fit_transform(df[['Première main']])

In [ ]:
# Appliquer le One-hot encoding pour les variables non-ordinales
df_encoded = pd.get_dummies(df[['Carburant', 'BoiteàV', 'Origine']], dtype=int)

# Supprimer les colonnes qui vont etre encoder
df = df.drop(['Carburant', 'BoiteàV', 'Origine'], axis= 1)

# Concatiner df avec df_encoded qui contient les colonnes encoder
df = pd.concat([df, df_encoded], axis= 1)
df

In [ ]:
# Supprimer les espaces d'apres le nom des variables
df.columns = df.columns.str.replace(" ", "_")

In [ ]:
df['Kilométrage'] = df['Kilométrage'].str.replace(" ", "")

# Remplacer l'intervalle du kilometrage par la valeur moyenne
def get_avrg(val):
    
    val = str(val)
    
    if '-' in val:
        bounds = val.split('-')
        return (int(bounds[0]) + int(bounds[1])) / 2
    
    elif 'Moins' in val:
        number = int(val.split('e')[1])
        return number / 2
    
    else:
        number = int(val.split('e')[1])
        return number / 2 + 10000

df['Kilométrage'] = df['Kilométrage'].apply(get_avrg)

In [ ]:
# Combiner la Marque et le modele 
df['Marque_Modele'] = (df['Marque'].str.strip() + '_' + df['Modèle'].str.strip()).str.replace(' ', '_')
df = df.drop(['Marque', 'Modèle'], axis= 1)

# Appliquer le target encoding pour remplacer chaque Marque_Modele par sa moyenne des prix
mean_prices = df.groupby('Marque_Modele')['Prix'].mean()

df['Marque_Modele_Encoded'] = df['Marque_Modele'].map(mean_prices)

In [ ]:
from gensim.models import FastText

# Gerer les lignes sans equipements
df['équipements'].fillna('', inplace= True)

# Gerer les carcteres dans equipements
df['équipements_clean'] = df['équipements'].str.lower().str.replace('/', ' ').str.replace(',', '').str.split()

sentences = df['équipements_clean'].tolist()

model = FastText(sentences, vector_size= 100, window= 3, min_count= 1 ,workers= 4)

In [ ]:
def vectorize_equipements(equipements_list, model):
    vectors = [model.wv[word] for word in equipements_list if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Appliquer le word embeding utilisant FastText au equipements
df['équipements_vector'] = df['équipements_clean'].apply(lambda x: vectorize_equipements(x, model))


In [ ]:
df = df.drop(['équipements', 'équipements_clean'], axis= 1)

In [ ]:
try:
    df.to_csv('voitures_preprocessed.csv', index=False, encoding='ISO-8859-1')
    print("file saved")
    
except:
    print("file didn't save")
